# Logging

로그 레벨은 아래와 같이 5개가 있다.

In [1]:
import logging

logging.debug('This is a debug message')
logging.info('This is an info message')
logging.warning('This is a warning message')
logging.error('This is an error message')
logging.critical('This is a critical message')

ERROR:root:This is an error message
CRITICAL:root:This is a critical message


기본적으로 WARNING 레벨 이상이 로그로 남게 되어 있다. 이에 대해서는 로그 레벨을 조정해 주면 됩니다.

A. basicConfig

1) basicConfig 함수를 이용하여 filename, level, filemode(a=append, w=기존의 내용을 삭제하고 새로작성), format
여러 설정을 한꺼번에 할 수 있습니다. 

2) basicConfig 함수를 사용하면 파이썬의 기본 root 로거를 사용한다.

In [1]:
import logging

logging.basicConfig(filename='app.log', level=logging.DEBUG, filemode='w', format='%(name)s - %(levelname)s - %(message)s')
logging.debug('This will get logged to a file')

B. 세부 설명

1) Logger: 로깅 클래스

2) LogRecord: Loggers(로깅 클래스)는 LogRecord 객체를 자동적으로 생성하고 
              LogRecord는 로그에 관련된 모든 정보를 가지고 있습니다. (라인넘버, 함수 등) 

3) Handler: 핸들러는 로그레코드를 로그가 저장되는 장소로 보냅니다.
            종류에는  StreamHandler, FileHandler, SMTPHandler, HTTPHandler 등이 있습니다.

4) Formatter: 포맷터는 로그를 남기는 형식을 지정할 수 있습니다.

In [3]:
import logging

logger = logging.getLogger('example_logger')
logger.warning('This is a warning')

위와 같이 커스텀 로거 example_logger를 생성하면 파이썬의 root 로거에 있는 기본 포맷터와 basicConfig를 사용할 수 없고, 따로 formatter와 handler를 정의해야 합니다. 
커스텀 로그 사용시 아래와 같이 "__name__" 파라미터를 getLogger() 로 보내는 것이 좋다고 합니다.  "__name__" 는 built-in 파이썬 변수로 현재 모듈의 이름을 저장하고 있습니다. 

In [1]:
import logging

# Create a custom logger
logger = logging.getLogger(__name__)

# Create handlers
c_handler = logging.StreamHandler()
f_handler = logging.FileHandler('file.log')
c_handler.setLevel(logging.WARNING)
f_handler.setLevel(logging.ERROR)

# Create formatters and add it to handlers
c_format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(c_handler)
logger.addHandler(f_handler)

logger.warning('This is a warning')
logger.error('This is an error')

__main__ - WARNING - This is a warning
__main__ - ERROR - This is an error


위와 같이 각각의 핸들러마다 다른 포맷과 저장장소, 로그 레벨을 줄 수 있습니다.
logger.warning(), logger.error() 은 로그 정보를 가진 LogRecord를 생성해서 c_handler와 f_handler로 보냅니다. 
c_handler는 StreamHandler로 WARNING 레벨이며 콘솔에 로그를 프린트합니다.
f_handler는 FileHandler로 ERROR 레벨이상만을 파일로 로그를 프린트 하고 ERROR 레벨 밑의 WARNING 등의 로그는 무시합니다.

C. 파일을 rotate 형식으로 할 경우

아래와 같이 logging, logging.handlers를 import 하여 RotatingFileHandler 를 이용하여 
해당 로그를 생성할 경로, 파일사이즈, 생성 파일 개수를 조정할 수 있습니다.

In [0]:
import logging
import logging.handlers as handlers

LOG = logging.getLogger(__name__)
logHandler = handlers.RotatingFileHandler("test.log",maxBytes=5000000, backupCount=5)
logHandler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s -%(name)s - %(levelname)s - %(message)s')
logHandler.setFormatter(formatter)
LOG.addHandler(logHandler)  
LOG.setLevel(logging.INFO)  
LOG.info("---------------start--------------------")

D. logging config 파일
1) config - xml 파일로 만드는 경우

In [0]:
[loggers]
keys=root,sampleLogger

[handlers]
keys=consoleHandler

[formatters]
keys=sampleFormatter

[logger_root]
level=DEBUG
handlers=consoleHandler

[logger_sampleLogger]
level=DEBUG
handlers=consoleHandler
qualname=sampleLogger
propagate=0

[handler_consoleHandler]
class=StreamHandler
level=DEBUG
formatter=sampleFormatter
args=(sys.stdout,)

[formatter_sampleFormatter]
format=%(asctime)s - %(name)s - %(levelname)s - %(message)s

In [0]:
import logging
import logging.config

logging.config.fileConfig(fname='file.conf', disable_existing_loggers=False)

# Get the logger specified in the file
logger = logging.getLogger(__name__)

logger.debug('This is a debug message')

In [ ]:
위와 같이 file.conf 설정 파일을 만들어 로깅 설정을 할 수도 있습니다. 
1) disable_existing_loggers 
False로 지정되면, 이 호출이 이루어졌을 때 존재하는 로거는 활성화된 상태로 남습니다.
기본값은 True

2) config-yaml인 경우

In [0]:
formatters:
  simple:
    format: '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
handlers:
  console:
    class: logging.StreamHandler
    level: DEBUG
    formatter: simple
    stream: ext://sys.stdout
loggers:
  sampleLogger:
    level: DEBUG
    handlers: [console]
    propagate: no
root:
  level: DEBUG
  handlers: [console]

In [0]:
import logging
import logging.config
import yaml

with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f.read())
    logging.config.dictConfig(config)

logger = logging.getLogger(__name__)

logger.debug('This is a debug message')